Marcin Wardyński  
czwartek, 8:00

In [1]:
from datasets import load_dataset
seed = 7
passage_number = 1000

corpus = load_dataset("clarin-knext/fiqa-pl", name="corpus")
passages = corpus['corpus'].shuffle(seed=seed).select(range(passage_number))['text']

/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Przy opracowywaniu prompt-a:
- użyj szablonu
- dopasuj treść prompta
- jeśli to konieczne użyj kontraprzykładu

Klasy NER-ów:
- PER
- ORG
- GPE
- LOC
- DATE

In [2]:
import requests
import hashlib
from diskcache import Cache
from functools import lru_cache

OLLAMA_SERVER_URL = "http://localhost:11434"
cache = Cache("ollama_cache")

@lru_cache(maxsize=1100)
def ask_ollama_with_memory_cache(model, prompt):
    return ask_ollama_with_disk_cache(model, prompt)

def ask_ollama_with_disk_cache(model, prompt):
    
    cache_key = hashlib.sha256((model+prompt).encode()).hexdigest()
    
    if cache_key in cache:
        answer = cache[cache_key]
        return (1, answer)
    else:
        answer = ask_ollama_server(model, prompt)
        cache[cache_key] = answer
        return (0, answer)


def ask_ollama_server(model, prompt):
    url = f"{OLLAMA_SERVER_URL}/api/generate"
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        data = response.json()
        return data["response"]

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None


In [46]:
!python -m spacy download pl_core_news_sm

Python(73480) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 20.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')


In [100]:
import pl_core_news_sm

nlp = pl_core_news_sm.load()

def find_named_entities_spacy(passage):
    results = {"persName": [],
                "orgName": [],
                "placeName": [],
                "geogName": [],
                "date": [],
                "time": []}

    doc = nlp(passage)
    for ent in doc.ents:
        results[ent.label_].append(str(ent))
    
    
    return results

In [102]:
find_named_entities_spacy(passages[0])

{'persName': ['G.', 'Edward Griffin'],
 'orgName': ['The Creature From Jekyll Island'],
 'placeName': [],
 'geogName': [],
 'date': [],
 'time': []}

In [17]:
prompt = "między znakami >>> i <<< znajduje się tekst, w którym masz odnaleźć nazwy własne i zaklasyfikować do jednej trzech kategorii: osoba, organizacja, miejsce, wynik przestaw jako json >>>„Chociaż jest tu kilka dobrych uwag dotyczących przyczyny inflacji (głównie związanych z podażą i popytem), azcoastal zmierza w innym kierunku, który ja sam zamierzałem obrać. Pozwólcie jednak, że podam inny punkt widzenia. Inną przyczyną inflacji jest drukowanie pieniędzy przez rząd (nie tylko zastąpienie starych pieniędzy nowymi, ale dodanie do całkowitej ilości pieniądza w obiegu).Jeśli rząd podwoi ilość pieniądza w obiegu (ze względu na argumenty i łatwą matematykę). ), wartość wszystkich pieniędzy spada dwukrotnie. To jest inflacja i jak to ujął G. Edward Griffin w The Creature From Jekyll Island, jest to tak naprawdę równoznaczne z ukrytym podatkiem. Krótko mówiąc, rząd federalny chce kupują fajne rzeczy, takie jak nowe czołgi lub samoloty, albo chcą dać kilka kartek żywnościowych biednym ludziom, albo chcą latać prywatnymi odrzutowcami, ale nie mają wystarczającej ilości pieniędzy z podatków. i wydawać je i kupować ich rzeczy. Ponieważ oni Właśnie zwiększyłem ilość pieniądza w obiegu, jednak pieniądz traci na wartości. Na przykład Twoje oszczędności straciły na wartości o połowę, pomimo tego, że na Twoim koncie oszczędnościowym znajduje się taka sama liczba dolarów. To tylko sposób, w jaki rząd może cię opodatkować bez nakładania na ciebie podatków. Kupują rzeczy, a ty masz teraz mniej pieniędzy (tzn. twoja emerytura jest mniej warta) i nawet nie wiesz, że właśnie zostałeś opodatkowany. Robi mi się niedobrze, że pozwoliliśmy naszym „przywódcom” ujść z tym na sucho”.'<<<"

ask_ollama_server(model, prompt)

' W podanym tekście nie ma takiej liczby nazw własnych, która mogłaby być sklasyfikowana do trzech kategorii: osoba, organizacja, miejsce. Jednak można wyodrębnić kilka nawiązujących do osób i instytucji:\n\n1. "G. Edward Griffin" – ten jest autorem książki "The Creature From Jekyll Island", która jest wymieniona w tekście.\n2. "rząd federalny" - to nawiązuje do instytucji rządowej, której działania są opisane w treści.\n3. W innych fragmentach tekstu można odnaleźć aluzje do innych osób i instytucji, ale bez imiennych wskaźników (np. "fajne rzeczy", które rząd chce kupić; "nowe czołgi lub samoloty" itp.).'

In [18]:
prompt = "między znakami >>> i <<< znajduje się tekst, w którym masz odnaleźć nazwy własne i zaklasyfikować do jednej z trzech kategorii: osoba, organizacja, miejsce, nazwa geograficzna, wynik przestaw jako json >>>'Zależy, co rozumiesz przez komunizm. Inną kwestią jest również to, czy dochód podstawowy jest opłacalny, czy nie. Nie mówię, że znam wszystkie odpowiedzi, ale dużo myślałem o przyszłości. Niektórzy obawiają się, że dochód podstawowy w zasadzie uczyniłby masy niewolnikami w pewien sposób, ponieważ nie rozwiąże nierówności w dochodach, sprawi, że ludzie będą zależni od pomocy rządowej. Inną opcją jest rozproszenie pracy. Zamiast pracować 40 godzin tygodniowo, pracują 30, 20 lub mniej. Ale to rozwiązanie może być tylko tymczasowe, dopóki ludzie nie będą po prostu nieopłacalni. Pojawi się pytanie, dlaczego pracujemy i po co tworzymy maszyny? Odciążyć ludzką pracę i służyć ludzkiemu życiu? Albo wydobyć jak najwięcej z Ziemi bez względu na ludzkie życie, zgromadzić wielki stos zasobów bez względu na to, czy istnieją ludzie, aby z nich korzystać.'<<<."

ask_ollama_server(model, prompt)

' {\n    "Osoba": ["Zależy"],\n    "Nazwa geograficzna": [],\n    "Organizacja": [],\n    "Miejsce": []\n}'

In [19]:
prompt = "między znakami >>> i <<< znajduje się tekst, w którym masz odnaleźć nazwy własne i zaklasyfikować do jednej z trzech kategorii: osoba, organizacja, miejsce, nazwa geograficzna, wynik przestaw jako json >>>'Zależy, co rozumiesz przez komunizm. Inną kwestią jest również to, czy dochód podstawowy jest opłacalny, czy nie. Nie mówię, że znam wszystkie odpowiedzi, ale dużo myślałem o przyszłości. Niektórzy obawiają się, że dochód podstawowy w zasadzie uczyniłby masy niewolnikami w pewien sposób, ponieważ nie rozwiąże nierówności w dochodach, sprawi, że ludzie będą zależni od pomocy rządowej. Inną opcją jest rozproszenie pracy. Zamiast pracować 40 godzin tygodniowo, pracują 30, 20 lub mniej. Ale to rozwiązanie może być tylko tymczasowe, dopóki ludzie nie będą po prostu nieopłacalni. Pojawi się pytanie, dlaczego pracujemy i po co tworzymy maszyny? Odciążyć ludzką pracę i służyć ludzkiemu życiu? Albo wydobyć jak najwięcej z Ziemi bez względu na ludzkie życie, zgromadzić wielki stos zasobów bez względu na to, czy istnieją ludzie, aby z nich korzystać.'<<<. Odpowiedz wyłącznie żądanym json i nic ponad to"

ask_ollama_server(model, prompt)

' {\n "osoba": ["ty"],\n "nazwa geograficzna": [],\n "miejsce": [],\n "organizacja": []\n}'

In [20]:
prompt = "Na podstawie tekstu między znakami >>> i <<< znajdź nazwy własne i zaklasyfikować do jednej z trzech kategorii: osoba, organizacja, miejsce: >>>'Zależy, co rozumiesz przez komunizm. Inną kwestią jest również to, czy dochód podstawowy jest opłacalny, czy nie. Nie mówię, że znam wszystkie odpowiedzi, ale dużo myślałem o przyszłości. Niektórzy obawiają się, że dochód podstawowy w zasadzie uczyniłby masy niewolnikami w pewien sposób, ponieważ nie rozwiąże nierówności w dochodach, sprawi, że ludzie będą zależni od pomocy rządowej. Inną opcją jest rozproszenie pracy. Zamiast pracować 40 godzin tygodniowo, pracują 30, 20 lub mniej. Ale to rozwiązanie może być tylko tymczasowe, dopóki ludzie nie będą po prostu nieopłacalni. Pojawi się pytanie, dlaczego pracujemy i po co tworzymy maszyny? Odciążyć ludzką pracę i służyć ludzkiemu życiu? Albo wydobyć jak najwięcej z Ziemi bez względu na ludzkie życie, zgromadzić wielki stos zasobów bez względu na to, czy istnieją ludzie, aby z nich korzystać.'<<<. Odpowiedz obiektem json w z polami: osoby, organizacje, miejsca"

ask_ollama_server(model, prompt)

' {\n "osoby": ["Osoba nieznana"],\n "organizacje": [],\n "miejsca": []\n}'

In [31]:
text = find_names_entities(passages[1:2])[0][0]
text

'Dzięki za artykuły z wiadomościami. Być może mógłbyś wziąć pod uwagę to [badanie Narodowego Instytutu Zdrowia](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1253709/), które przyglądało się badaniu NHS, które wciąż przytaczasz? W szczególności >Ekspozycja na glifosat nie była związana z ogólną zachorowalnością na raka ani z większością badanych przez nas podtypów raka. Sugerowano związek z występowaniem szpiczaka mnogiego [raka komórek plazmatycznych], który należy monitorować, ponieważ więcej przypadków występuje w AHS. Czy nie należy tego również brać pod uwagę, gdy mówimy o tym, czy Round-up powoduje raka? W tym momencie nauka wydaje się daleka od ugruntowania. Wszystko, co połączyłeś, dotyczy chłoniaka nieziarniczego. Rozumiem, dlaczego Monsanto zignorowało dane, które wskazują na problem, i skupiło się na tym pojedynczym, właśnie analizowanym badaniu. Dlaczego jesteś?'

In [56]:
prompt = "Przeanalizuj podany tekst i wypisz nazwy własne podzielone na kategorie: osoby, organizacje, miejsca. Twoja odpowiedź powinna składać się wyłącznie z obiektu JSON o formacie zdefiniowanym poniżej. Jeśli pewna kategoria nie ma odnalezionych reprezentantów, zwróć dla niej pustą tablicę. Upewnij się, że wynik jest czysty i zawiera wyłącznie obiekt JSON\n" +\
"Format JSON:\n" +\
"{'osoba': [\"lista osób\"],  \"organizacja\": [\"lista organizacji\"], \"miejsce\": [\"lista miejsc\"]}\n"+\
"Tekst wejściowy:\n"+\
"\""+text+"\"\n"+\
"Wynik:"

ask_ollama_server(model_mistral, prompt)

' {\n    "osoba": [],\n    "organizacja": ["Narodowy Instytut Zdrowia", "Monsanto"],\n    "miejsce": []\n  }'

In [ ]:
import json
import time

start_time = time.time()

for passage in passages:

    prompt = "Przeanalizuj podany tekst i wypisz nazwy własne podzielone na kategorie: osoby, organizacje, nazwy administracyjne miejsc, nazwy geograficzne, daty, czas. Twoja odpowiedź powinna składać się wyłącznie z obiektu JSON o formacie zdefiniowanym poniżej. Jeśli pewna kategoria nie ma odnalezionych reprezentantów, zwróć dla niej pustą listę. Upewnij się, że wynik jest czysty i zawiera wyłącznie obiekt JSON\n" +\
    "Format JSON:\n" +\
    "{'persName': [\"lista osób\"],  \"orgName\": [\"lista organizacji\"], \"placeName\": [\"lista nazw administracyjnych miejsc\"], \"geog\": [\"lista nazw geograficznych\"], \"date\": [\"lista dat\"], \"time\": [\"lista określeń czasu\"]}\n"+\
    "Tekst wejściowy:\n"+\
    "\""+passage+"\"\n"+\
    "Wynik:"

    ask_ollama_with_disk_cache(model_mistral, prompt)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time}s")




Elapsed time: 3257.7920219898224s


In [92]:
prompt_intro = "Przeanalizuj podany tekst i wypisz nazwy własne podzielone na kategorie: osoby, organizacje, nazwy administracyjne miejsc, nazwy geograficzne, daty, czas. Twoja odpowiedź powinna składać się wyłącznie z obiektu JSON o formacie zdefiniowanym poniżej. Jeśli pewna kategoria nie ma odnalezionych reprezentantów, zwróć dla niej pustą listę. Upewnij się, że wynik jest czysty i zawiera wyłącznie obiekt JSON\n"
prompt_json_format = "Format JSON:\n{'persName': [\"lista osób\"],  \"orgName\": [\"lista organizacji\"], \"placeName\": [\"lista nazw administracyjnych miejsc\"], \"geog\": [\"lista nazw geograficznych\"], \"date\": [\"lista dat\"], \"time\": [\"lista określeń czasu\"]}\n"
prompt_output = "Wynik:"

In [96]:
start_time = time.time()

for passage in passages:
    prompt_input = f"Tekst wejściowy:\n\"{passage}\"\n"
    prompt = prompt_intro + prompt_json_format + prompt_input + prompt_output
        
    ask_ollama_with_disk_cache(model_mistral, prompt)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time}s")

Elapsed time: 3200.7369158267975s


In [93]:
example1_text = "Czy możesz mi pokazać gdzie? Ponieważ z pewnością, kiedy Obama nałożył podatek na finansowanie ACA, gospodarka rozkwitła, zatrudniono milion lekarzy, setki szpitali i tysiące spacerów w klinikach, które zbudowaliśmy. A giełda rozwijała się bardziej."
example1_answer = "{'persName': [\"Obama\"],  \"orgName\": [\"ACA\"], \"placeName\": [], \"geogName\": [], \"date\": [], \"time\": []}"

example2_text = "„Postęp komputerowy w coraz większym stopniu pozwala producentom dostosowywać zamówienia i szybciej wysyłać towary. W nowym świecie wytwarzanie produktów w odległych krajach o niskich płacach, takich jak Chiny, może być wadą: wysyłanie gotowych produktów do Stanów Zjednoczonych może trwać zbyt długo – tygodnie, miesiące. „Chodzi o bliskość z klientem” – powiedział Michael Mandel, główny strateg ekonomiczny w Progressive Policy Institute. „„Zdobędziesz trwałą i trwałą przewagę nad konkurencją zagraniczną”. który okupował Biały Dom. Mimo to prezydent Donald Trump skorzystał z okazji, by w środę wziąć udział w ogłoszeniu przez Foxconna, mówiąc, że „zdecydowanie” wydarzyło się z powodu jego wyboru i dążenia do cięć podatkowych i regulacyjnych. [źródło] ://hosted.ap.org/dynamic/stories/U/US_BC_US_AMAZON_AND_FOXCONN_SPEED_TO_CUSTOMERS?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT&CTIME=2017-07-27-03-16-36)\""
example2_answer = "{'persName': [\"Michael Mandel\", \"Donald Trump\"],  \"orgName\": [\"Progressive Policy Institute\", \"Biały Dom\", \"Foxconna\"], \"placeName\": [\"Chiny\", \"Stanów Zjednoczonych\"], \"geogName\": [], \"date\": [], \"time\": []}"

example3_text = "„Płać za grę? Masz na myśli, że płacą za mocniejsze źródła danych, których inne firmy nie potrzebują, na pewno tak. Ale każdy może teraz handlować na giełdach amerykańskich (NYSE, NASDAQ, BATS itp.), co nie było prawdą 20 lat temu kiedy NYSE miała specjalistyczny monopol, więc tak, rynki są bardziej demokratyczne niż kiedykolwiek. Uwaga na marginesie, giełdy czerpią bezpośrednie zyski ze zwiększonego wolumenu obrotu, ponieważ przyjmują niewielki procent każdej transakcji, więc nie jestem pewien, co to znaczy, że nie czerpią z tego bezpośrednich korzyści. Mam też wrażenie, że nie rozumiesz zakresu działalności HFT, szczytowe zyski HFT były rzędu 7 miliardów w czasie największego wolumenu w ostatniej dekadzie (2005-2010 ). Są teraz około 1B. W porównaniu z bilionami dolarów, które zmieniają właściciela w ciągu roku, jest to kiepska wymiana, trudno nazwać „kran z darmowymi pieniędzmi”. Również Katsayuma otworzyła kolejną ciemną pulę, która obsługuje duże wolumeny klientów (twoich goldmans i merrils) Jedyną różnicą w IEX jest to, że ma darmową kampanię marketingową ign, aby przyciągnąć klientów. Poważnie, IEX jest niczym innym jak istniejącymi stałymi, krzyżowymi ciemnymi basenami, które przy okazji wkurzają inwestorów detalicznych bardziej niż oświetlone giełdy, takie jak NASDAQ. Katsayuma został zmiażdżony w swoich egzekucjach, ponieważ nie mógł nadążyć z czasem, a potem trafił w dziesiątkę, namawiając Michaela Lewisa, by namalował go jako „bohatera”, szczerze mówiąc, jestem zdziwiony, jakie szczęście miał ten koleś. Dealerzy brokerów BTW otrzymują preferencyjne traktowanie w IEX, co oznacza, że ​​mogą ciąć przed inwestorami detalicznymi. Dlaczego tak się na to zastanawiasz, czy wykonałeś kilka złych transakcji na eTrade i potrzebujesz kozła ofiarnego?”"
example3_answer = "{'persName': [\"Katsayuma\", \"Katsayuma\", \"Michaela Lewisa\"],  \"orgName\": [\"NYSE\", \"NASDAQ\", \"BATS\", \"NYSE\", \"HFT\", \"goldmans\", \"merrils\", \"IEX\", \"ign\", \"IEX\", \"NASDAQ\", \"IEX\", \"eTrade\"], \"placeName\": [], \"geogName\": [], \"date\": [\"2005\", \"2010\"], \"time\": []}"
example2_full = f"Przykład 3:\n\"{example2_text}\"\nOdpowiedź do przykładu 1:\n\"{example2_answer}\"\n"

example4_text = "Nie zaczynaj od inwestowania w kilka pojedynczych firm. To ryzykowne. Chcesz przykładu? Myślę o dużej firmie, powiedzmy około 120 miliardów dolarów, znanej firmie i dobrych stałych dywidendach. Radzili sobie całkiem nieźle i generalnie byli zajęci przekonywaniem ludzi, że patrzą w przyszłość z nowymi, przyjaznymi dla środowiska technologiami. Potem... poszli i wylali garść ropy do Zatoki Meksykańskiej. Tak, to nie był ładny obraz, gdyby BP było tego dnia jedną z pięciu spółek w twoim portfelu. Sprawy wyglądałyby jednak znacznie lepiej, gdyby byli jedną z 500 lub 5000 firm. Więc. Po pierwsze, dążyć do dywersyfikacji za pośrednictwem funduszy inwestycyjnych lub ETF. (Osobiście uważam, że prawdopodobnie powinieneś zacząć od funduszy wzajemnych: po pierwsze unikasz opłat transakcyjnych. Łatwiej jest również umieścić średnie kwoty w dolarach w funduszach niż w ETF-ach, nawet jeśli otrzymujesz wolny od opłat handel ETF-ami. może dać ci lepsze wskaźniki wydatków, ale im mniej pieniędzy zainwestowałeś, tym mniej ważne.) Gdy masz przyzwoity portfel – dziesiątki tysięcy dolarów lub więcej – możesz zacząć rozważać posiadanie akcji poszczególnych spółek. Zwróć uwagę na opłaty, w tym opłaty transakcyjne / prowizje. Jeśli kupisz akcje o wartości 2000 USD i zapłacisz 20 USD prowizji, stracisz już 1%. Jeśli trzymasz fundusz powierniczy lub ETF, spójrz na wskaźnik wydatków. Roczna realna stopa zwrotu na giełdzie wynosi około 4%. (Prawdziwy zwrot jest po uwzględnieniu inflacji.) Jeśli Twoja opłata wynosi 1%, to około jedna czwarta Twoich zarobków, co jest ogromne. I chociaż funduszowi powierniczemu łatwo jest od czasu do czasu przewyższyć rynek o 1%, to naprawdę ciężko jest robić to konsekwentnie. Kiedy już przyjrzysz się poszczególnym firmom, powinieneś przeprowadzić wiele nieznośnych, nudnych, głupich poszukiwań, a nie kupować tylko akcji na podstawie ich marki. Będziesz zainteresowany kilkoma danymi. Głównym z nich jest prawdopodobnie wskaźnik P/E (cena/zysk). Jeśli weźmiesz odwrotność tego, uzyskasz wskaźnik, w jakim Twoja inwestycja przyniesie Ci pieniądze (np. P/E 20 to 5%, P/E 10 to 10%). Jeśli wszystko inne jest równe, niższy wskaźnik P/E jest dobrą rzeczą: oznacza to, że kupujesz dochód firmy naprawdę tanio. Jednak wszystko inne rzadko jest równe: jeśli akcje są naprawdę tanie, zwykle dzieje się tak dlatego, że inwestorzy nie myślą, że mają przed sobą długą przyszłość. Zarobki nie zawsze są spójne. Istnieje wiele innych miar, takich jak beta (korelacja z ogólnym rynkiem: bardziej ryzykowne akcje niestabilne mają wyższe wartości), marże brutto, cena do nielewarowanych wolnych przepływów pieniężnych i tym podobne. Ponownie wykonaj nudne badania, w przeciwnym razie po prostu grasz w gry ze swoimi pieniędzmi."
example4_answer = "{'persName': [],  \"orgName\": [\"BP\"], \"placeName\": [], \"geogName\": [\"Zatoki Meksykańskiej\"], \"date\": [], \"time\": []}"

example5_text = "„To najlepszy tl;dr, jaki mogłem zrobić, [oryginał](https://www.bloomberg.com/news/articles/2017-08-22/hong-kong-braces-for-storm-hato-stock- handel może zostać zakłócony) zmniejszony o 73% (jestem botem) ***** > Hongkong podniósł poziom ostrzeżenia przed burzą do najwyższego poziomu po raz pierwszy od pięciu lat i odwołał poranną sesję handlową jako Poważna Tajfun Hato zbliżył się do centrum finansowego.> Jeśli sygnał będzie obowiązywał do południa, handel na czwartym co do wielkości rynku akcji na świecie zostanie dziś zlikwidowany, zgodnie z zasadami Hong Kong Exchanges & Clearing Ltd. > O godzinie 9 rano ciężki tajfun Hato znajdował się około 80 kilometrów na południe od Hongkongu, podało Obserwatorium.***** [**Rozszerzone podsumowanie**](http://np.reddit.com/r/autotldr/ komentarze/6vgq2t/hong_kong_delays_morning_trading_as_typhoon_hato/) | [FAQ](http://np.reddit.com/r/autotldr/comments/31b9fm/faq_autotldr_bot/ \"\"Wersja 1.65, ~196625 tl\");[do tej pory drs.\" Opinia](http://np.reddit.com/message/compose?to=%23autotldr \"\"PM i komentarze są monitorowane, mile widziane są konstruktywne opinie.\"\") | *Najlepsze* *Słowa kluczowe*: **Hong**^#1 **Kong**^#2 **Zamknij**^#3 **Tajfun**^#4 **Handel**^#5\""
example5_answer = "{'persName': [],  \"orgName\": [\"Hong Kong Exchanges & Clearing Ltd.\"], \"placeName\": [\"Hongkong\", \"Hongkongu\"], \"geogName\": [], \"date\": [], \"time\": [\"9 rano\"]}"

examples_text = [example1_text, example2_text, example3_text, example4_text, example5_text]
examples_answer = [example1_answer, example2_answer, example3_answer, example4_answer, example5_answer]


examples_snippet = []
for i in range(len(examples_text)):
    examples_snippet.append(f"Przykład {i+1}:\n\"{examples_text[i]}\"\nOdpowiedź do przykładu {i+1}:\n\"{examples_answer[i]}\"\n")

In [97]:
def perform_few_shot_prompt(passage):
    prompt_input = f"Tekst wejściowy:\n\"{passage}\"\n"
    prompt = prompt_intro + prompt_json_format + "".join(examples_snippet) + prompt_input + prompt_output
        
    result = ask_ollama_with_disk_cache(model_mistral, prompt)
    return json.loads(result)
    
start_time = time.time()

for passage in passages:
    perform_few_shot_prompt(passage)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time}s")

Elapsed time: 10250.866141080856s


In [ ]:
#does single entry occures multiple times, like in the text

def calculate_conf_matrix(baseline_entities, other_entities):
    base_ents_tmp = baseline_entities[:]
    other_ents_tmp = other_entities[:]

    tp_list = []

    for base_ent in baseline_entities:
        if base_ent in other_ents_tmp:
            base_ents_tmp.remove(base_ent)
            other_ents_tmp.remove(base_ent)
            tp_list.append(base_ent)

    tp = len(tp_list)
    fp = len(other_ents_tmp)
    fn = len(base_ents_tmp)

    return tp, fp, fn

def calculate_precision_recall_f1_score(conf_matrix_by_class):
    results = {}
    for entity_class, conf_matrix in conf_matrix_by_class.items():
        results[entity_class] = {}

        precision = conf_matrix["TP"]/(conf_matrix["TP"]+conf_matrix["FP"])
        results[entity_class]["precision"] = precision

        recall = conf_matrix["TP"]/(conf_matrix["TP"]+conf_matrix["FN"])
        results[entity_class]["recall"] = recall

        f1_score = 2 * (precision * recall) / (precision + recall)
        results[entity_class]["f1_score"] = f1_score

def calculate_metrics(passages, baseline_fun, other_fun):
    conf_matrix_by_class = {"persName": {"TP": 0, "FP": 0, "FN": 0},
                                "orgName": {"TP": 0, "FP": 0, "FN": 0},
                                "placeName": {"TP": 0, "FP": 0, "FN": 0},
                                "geogName": {"TP": 0, "FP": 0, "FN": 0},
                                "date": {"TP": 0, "FP": 0, "FN": 0},
                                "time": {"TP": 0, "FP": 0, "FN": 0},
                                "total": {"TP": 0, "FP": 0, "FN": 0}}
    
    for passage in passages:
        results_baseline = baseline_fun(passage)
        results_other = other_fun(passage)

        for entity_class in results_baseline.keys():
            baseline_entities = results_baseline[entity_class]
            other_entities = results_other[entity_class]

            tp, fp, fn = calculate_conf_matrix(baseline_entities, other_entities)

            conf_matrix_by_class[entity_class]["TP"] += tp
            conf_matrix_by_class["total"]["TP"] += tp
            conf_matrix_by_class[entity_class]["FP"] += fp
            conf_matrix_by_class["total"]["FP"] += fp
            conf_matrix_by_class[entity_class]["FN"] += fn
            conf_matrix_by_class["total"]["FN"] += fn

    return calculate_precision_recall_f1_score(conf_matrix_by_class)

calculate_metrics(passages, )

Czasy na przejście po wszystkich pasażach:

Zero-Shot: 3200.7369158267975s ~53:20
Few-Shot: 10250.866141080856s 2:50:50

#### Porównanie NER do LLM NER

Czas, pamięć, miejsce na HDD, miejsce w pamięci i obciążenie CPU